<a href="https://colab.research.google.com/github/DocBrown85/AppliedDataScienceCapstone/blob/main/Neighborhoods_Toronto_Part2_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Segmenting Toronto Neighborhoods

## Importing Dependencies

In [2]:
!pip3 install beautifulsoup4
!pip install geocoder

     |████████████████████████████████| 102kB 7.5MB/s 


In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from bs4 import BeautifulSoup


# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


<a id='item1'></a>


## Scrape the Dataset


In [18]:
URL = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&direction=prev&oldid=926287641'
page = requests.get(URL)

soup = BeautifulSoup(page.content, 'html.parser')
table_contents=[]
table=soup.find('table')
for row in table.findAll("tr"):
    cell = {}
    cells = row.findAll("td")
    if len(cells) == 0:
      continue
    if 'Not assigned' in cells[1]:
      pass
    else:
      cell['PostalCode'] = cells[0].text[:3]
      cell['Borough'] = (cells[1].text).split('(')[0]
      cell['Neighborhood'] = cells[2].text.strip()
      table_contents.append(cell)

#print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
neighborhoods_bare=df.groupby(df['PostalCode'], as_index=False).agg(lambda x: ', '.join(set(x.dropna())))

In [19]:
neighborhoods_bare.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Morningside, Guildwood, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [20]:
neighborhoods.shape

(103, 3)

## Add Latitute and Longitude

In [17]:
url = "https://raw.githubusercontent.com/DocBrown85/AppliedDataScienceCapstone/main/Geospatial_Coordinates.csv"
postal_codes_geo_coordinates = pd.read_csv(url)
postal_codes_geo_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [23]:
neighborhoods = pd.merge(neighborhoods_bare, postal_codes_geo_coordinates, left_on=["PostalCode"], right_on =["Postal Code"])
neighborhoods.head()

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Morningside, Guildwood, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476
